In [11]:
import pandas as pd
import sqlite3
import requests
from bs4 import BeautifulSoup

In [3]:
conn = sqlite3.connect(':memory:') 
cursor = conn.cursor()
cursor.execute('CREATE TABLE students (id INTEGER, name TEXT, grade INTEGER)')
data = [(1, 'Alice', 85), (2, 'Bob', 90), (3, 'Charlie', 78), (4, 'David', 92)]
cursor.executemany('INSERT INTO students VALUES (?,?,?)', data)
conn.commit()

In [5]:
# data collection from a database
query = "SELECT * FROM students"
df_db = pd.read_sql_query(query, conn)
print(df_db)
conn.close()

   id     name  grade
0   1    Alice     85
1   2      Bob     90
2   3  Charlie     78
3   4    David     92


In [7]:
# data collection from an API

api_url = "https://jsonplaceholder.typicode.com/users"
response = requests.get(api_url)

if response.status_code == 200:
    json_data = response.json()
    df_api = pd.DataFrame(json_data)
    df_api_clean = df_api[['id', 'name', 'email']]
    print(df_api_clean.head())
else:
    print("Failed to fetch API data")

   id              name                      email
0   1     Leanne Graham          Sincere@april.biz
1   2      Ervin Howell          Shanna@melissa.tv
2   3  Clementine Bauch         Nathan@yesenia.net
3   4  Patricia Lebsack  Julianne.OConner@kory.org
4   5  Chelsey Dietrich   Lucio_Hettinger@annie.ca


In [9]:
# data colletion from a sensor

sensor_data_url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv"

try:
    df_sensor = pd.read_csv(sensor_data_url, parse_dates=['Date'])
    print(df_sensor.head())
except Exception as e:
    print(f"Error: {e}")

        Date  Temp
0 1981-01-01  20.7
1 1981-01-02  17.9
2 1981-01-03  18.8
3 1981-01-04  14.6
4 1981-01-05  15.8


In [13]:
# data collection by web scraping

url = "http://quotes.toscrape.com/"

try:
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    # The site holds quotes in <span class="text"> and authors in <small class="author">
    quotes = soup.find_all('span', class_='text')
    authors = soup.find_all('small', class_='author')

    scraped_data = []
    for quote, author in zip(quotes, authors):
        scraped_data.append({
            'Author': author.text,
            'Quote': quote.text
        })

    df_scraping = pd.DataFrame(scraped_data)
    print(df_scraping.head())

except Exception as e:
    print(f"Error: {e}")

            Author                                              Quote
0  Albert Einstein  “The world as we have created it is a process ...
1     J.K. Rowling  “It is our choices, Harry, that show what we t...
2  Albert Einstein  “There are only two ways to live your life. On...
3      Jane Austen  “The person, be it gentleman or lady, who has ...
4   Marilyn Monroe  “Imperfection is beauty, madness is genius and...


In [15]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from io import BytesIO
# collecting an image

# Using a standard data science sample image (more reliable than Wikipedia for testing)
image_url = "https://raw.githubusercontent.com/scikit-image/scikit-image/main/skimage/data/chelsea.png"

try:
    # We use a specific header to look like a real browser
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'}
    response = requests.get(image_url, headers=headers, stream=True)
    
    # Check if the download was actually successful (Code 200)
    if response.status_code == 200:
        img = Image.open(BytesIO(response.content))
        img_array = np.array(img)
        
        print(f"Image Shape: {img_array.shape}")
        print(f"Data Type: {img_array.dtype}")
        
        plt.figure(figsize=(4, 4))
        plt.imshow(img_array)
        plt.axis('off')
        plt.show()
    else:
        print(f"Failed to download. Status Code: {response.status_code}")

except Exception as e:
    print(f"Error: {e}")

Failed to download. Status Code: 404


In [27]:
import time
from datetime import datetime

# collecting data from a live sensor

# URL for USGS live earthquake sensor feed (GeoJSON format)
# This feed contains all earthquakes recorded in the last hour
URL = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_hour.geojson"

def get_live_sensor_data():
    try:
        # 1. Extract: Send a GET request to the cloud endpoint
        response = requests.get(URL)
        
        if response.status_code == 200:
            # 2. Parse: Convert JSON response to a Python dictionary
            data = response.json()
            
            # The 'features' list contains the actual sensor readings
            readings = data['features']
            
            print(f"\n--- Live Sensor Update: {datetime.now().strftime('%H:%M:%S')} ---")
            print(f"Total events detected in past hour: {data['metadata']['count']}")
            
            # Display the first 3 most recent readings
            for i, reading in enumerate(readings[:3]):
                props = reading['properties']
                magnitude = props['mag']
                location = props['place']
                print(f"#{i+1}: Magnitude {magnitude} - Location: {location}")
        else:
            print("Error: Could not connect to cloud.")
            
    except Exception as e:
        print(f"An error occurred: {e}")
        # Run the extraction loop
if __name__ == "__main__":
    print("Starting Live Sensor Stream...")
    get_live_sensor_data()

Starting Live Sensor Stream...

--- Live Sensor Update: 12:37:44 ---
Total events detected in past hour: 19
#1: Magnitude 2.69 - Location: 20 km NNE of Indio, CA
#2: Magnitude 0.83 - Location: 21 km NNE of Indio, CA
#3: Magnitude 1.61 - Location: 18 km N of Indio, CA


In [29]:
import json
# collecting data in json format and also flattening it

json_str = """
{
    "model_name": "GPT-X",
    "version": 4.0,
    "layers": [
        {"id": 1, "type": "attention", "units": 512},
        {"id": 2, "type": "feed_forward", "units": 2048}
    ],
    "metrics": {
        "accuracy": 0.95,
        "loss": 0.05
    }
}
"""

data = json.loads(json_str)

# Flatten 'layers' while keeping model-level fields
df = pd.json_normalize(
    data,
    record_path=['layers'],
    meta=['model_name', 'version', ['metrics', 'accuracy'], ['metrics', 'loss']]
    )

print(df)

   id          type  units model_name version metrics.accuracy metrics.loss
0   1     attention    512      GPT-X     4.0             0.95         0.05
1   2  feed_forward   2048      GPT-X     4.0             0.95         0.05


In [31]:
import xml.etree.ElementTree as ET
# collecting data in xml format and also parsing it

xml_data = """
<dataset>
    <record>
        <id>101</id>
        <feature>Temperature</feature>
        <value>23.5</value>
    </record>
    <record>
        <id>102</id>
        <feature>Humidity</feature>
        <value>45.2</value>
    </record>
</dataset>
"""

# Parse XML
root = ET.fromstring(xml_data)

# Process: Extract data into a list of dictionaries
parsed_data = []
for record in root.findall('record'):
    r_id = record.find('id').text
    feature = record.find('feature').text
    value = float(record.find('value').text)
    parsed_data.append({
        "ID": r_id,
        "Feature": feature,
        "Value": value
    })

# Convert into DataFrame
df_xml = pd.DataFrame(parsed_data)

print(df_xml)

    ID      Feature  Value
0  101  Temperature   23.5
1  102     Humidity   45.2


In [35]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
# data collection from a dynamic website using Selenium

url = "http://quotes.toscrape.com/js/"

# 1. Setup Headless Chrome (Runs without a UI window)
chrome_options = Options()
chrome_options.add_argument("--headless") 
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

data_selenium = []
try:
    # 2. Initialize the driver
    print("Initializing browser...")
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    
    # 3. Load the page
    print(f"Accessing {url}...")
    driver.get(url)

    # 4. CRITICAL: Wait for JavaScript to load content
    # We wait up to 10 seconds for the class 'quote' to appear
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "quote"))
    )
    
    # 5. Extract Data
    quotes = driver.find_elements(By.CLASS_NAME, "quote")
    
    for q in quotes:
        text = q.find_element(By.CLASS_NAME, "text").text
        author = q.find_element(By.CLASS_NAME, "author").text
        data_selenium.append({"Author": author, "Quote": text})
        
    # Convert to DataFrame
    df_dynamic = pd.DataFrame(data_selenium)
    print(df_dynamic.head())

except Exception as e:
    print(f"Selenium Error: {e}")

finally:
    # 6. Always quit the driver to free resources
    if 'driver' in locals():
        driver.quit()

Initializing browser...
Accessing http://quotes.toscrape.com/js/...
            Author                                              Quote
0  Albert Einstein  “The world as we have created it is a process ...
1     J.K. Rowling  “It is our choices, Harry, that show what we t...
2  Albert Einstein  “There are only two ways to live your life. On...
3      Jane Austen  “The person, be it gentleman or lady, who has ...
4   Marilyn Monroe  “Imperfection is beauty, madness is genius and...
